# chapter 6 다양한 데이터 타입 다루기

* boolen type
* 수치 
* 문자열
* data와 timestamp
* null
* 복합 데이터
* 사용자 정의 함수

* **DataFrame(Dataset) 메서드**
        DataFrame은 Row 타입을 가진 Dataset이므로 결국에는 Dataset메서드를 만나게 된다. DataFrameStatFunctions와 DataFrameNaFunctions 등 Dataset의 하위 모듈은 다양한 메서드를 제공한다. 이 메서드를 사용해 여러가지 문제를 해결할 수 있다.

* **Column 메서드**
        Column은 alias나 contains 같이 컬럼과 관련된 여러가 메서드를 제공한다.

In [ ]:
from pyspark.sql import SparkSession # SparkSession: 스파크 코드를 실행하기 위한 진입점

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
df = spark.read.format("csv")\
    .option("header","true")\
    .option('inferSchema','true')\
    .load("/Users/taewoong/Documents/coding/Spark_practice/data/retail-data/by-day/2010-12-01.csv")

df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



### 스파크 데이터 타입으로 변환하기

프로그래밍 연어의 고유 데이터 타입을 스파크 데이터 타입으로 변환해보자. 스파크 데이터 차입으로 변환하는 방법은 반드시 알아두어야 한다. 데이터 타입 면환은 lit함수를 사용한다. lit함수는 다른 언어의 데이터 타입을 스파크 데이터 타입에 맞게 변환한다. 

In [6]:
from pyspark.sql.functions import lit

In [7]:
df.select(lit(5), lit("five"),lit(5.0)),

(DataFrame[5: int, five: string, 5.0: double],)

### boolen type 다루기

불리언 구문은 and, or, true, false로 구성된다. 

In [8]:
from pyspark.sql.functions import col


In [11]:
df.where(col('InvoiceNo')!= 536365).select('InvoiceNo',"Description").show(5,False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



가장 명확한 방법은 문자열 표현식에 조건절을 명시하는 것이다.

In [12]:
df.where("InvoiceNo = 536365").show(5,False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [13]:
df.where("InvoiceNo <> 536365").show(5,False)

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT|32      |2010-12-01 08:34:00|1.69     |13047.0   |United Kingdom|
|536367   |22745    |POPPY'S PLAYHOUSE BEDROOM    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
|536367   |22748    |POPPY'S PLAYHOUSE KITCHEN    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
+---------+---------+-----------------------------+--------+----

and 메서드나 or메서드를 사용해서 불리언 표현식을 여러 부분에 지정할 수 있다. 불리언 표현식을 사용하는 경우 항상 모든 표현식을 and메서드로 묶어 차례대로 필터를 적용해야 한다.<br>
불리언 문을 차례대로 표현하더라도 스파크는 내부적으로 and 구문을 필터 사이에 추가해 모든 필터를 하나의 문장으로 변환한다. 그런 다음 동시에 모든 필터를 처리한다. and구문으로 저건문을 만들 수 있다. 하지만 차례로 조건을 나열하면 이하해기 쉽고 읽기도 편해다. 반면 or구문을 사용할 때는 반드시 구문에 조건을 정의해애 한다.

In [14]:
from pyspark.sql.functions import instr # 검색 문자열에서 특정 패턴의 위치를 식별하십시오.

In [17]:
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description,"POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



불리언 컬럼을 사용해 DataFrame을 필터링할 수도 있다.

In [20]:
DOTCodeFilter = col("StockCode") == "DOT"

df.withColumn("isExpensive",DOTCodeFilter & (priceFilter|descripFilter))\
    .where("isExpensive")\
    .select("unitPrice","isExpensive").show()

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



불리언 표현식을 만들때 null값 데이터는 다르게 처리해야 한다. null값에 안전한 동치테스트를 수행한다.

In [21]:
df.where(col('Description').eqNullSafe('hellow')).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



### 수치형 데이터 타입 다루기

* pow함수는 표시된 지수만큼 컬럼의 값을 거듭제곱한다.

In [22]:
from pyspark.sql.functions import expr, pow

In [23]:
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"),2) +5

In [24]:
df.select(expr("CustomerId"),fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [25]:
df.selectExpr(
    "CustomerId",
    "(POWER((Quantity * UnitPrice),2.0)+5) as realQuantity"
).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



* 반올림도 자주 사용하는 수치형 작업중 하나이다. 

In [27]:
from pyspark.sql.functions import lit, round, bround

df.select(round(lit("2.5")),bround(lit("2.5"))).show(5)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 5 rows



In [28]:
from pyspark.sql.functions import corr

In [30]:
df.stat.corr("Quantity","UnitPrice")

-0.04112314436835551

In [31]:
df.select(corr("Quantity","UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [32]:
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

* approxQuantile 메서드 : 데이터의 백분위수 계산, 근사치

In [34]:
olName = "UnitPrice"
quantileProbs=[0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice",quantileProbs,relError)

[2.51]

StateFunctions 패키지는 교차표나 자주 사용하는 항목 쌍을 확인하는 용도의 메서드도 제곡한다.

In [37]:
df.stat.crosstab("StockCode","Quantity").show(5)

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

monotonically_increasing_id : 모든 로우에 고유 ID 값을 추가한다.

In [38]:
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show(10)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
|                            9|
+-----------------------------+
only showing top 10 rows



### 문자열 데이터 타입 다루기


* 대/소문자 변환 처리 작업 initcap함수 사용

In [40]:
from pyspark.sql.functions import initcap

In [41]:
df.select(initcap(col("Description"))).show()

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
|Set 7 Babushka Ne...|
|Glass Star Froste...|
|Hand Warmer Union...|
|Hand Warmer Red P...|
|Assorted Colour B...|
|Poppy's Playhouse...|
|Poppy's Playhouse...|
|Feltcraft Princes...|
|Ivory Knitted Mug...|
|Box Of 6 Assorted...|
|Box Of Vintage Ji...|
|Box Of Vintage Al...|
|Home Building Blo...|
|Love Building Blo...|
|Recipe Box With M...|
+--------------------+
only showing top 20 rows



* lower, upper

In [43]:
from pyspark.sql.functions import lower, upper

In [45]:
df.select(col("Description"),
         lower(col("Description")),
         upper(lower(col("Description")))).show(5)

+--------------------+--------------------+-------------------------+
|         Description|  lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
|WHITE HANGING HEA...|white hanging hea...|     WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern|      WHITE METAL LANTERN|
|CREAM CUPID HEART...|cream cupid heart...|     CREAM CUPID HEART...|
|KNITTED UNION FLA...|knitted union fla...|     KNITTED UNION FLA...|
|RED WOOLLY HOTTIE...|red woolly hottie...|     RED WOOLLY HOTTIE...|
+--------------------+--------------------+-------------------------+
only showing top 5 rows



* lpad, ltrim, rpad, rtrim, trim : 문자열 주변의 공백을 제거하거나 추가하는 메서드

In [46]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim

In [47]:
df.select(
    ltrim(lit("    HELLO   ")).alias("ltrim"),
    rtrim(lit("    HELLO   ")).alias("rtrim"),
    trim(lit("    HELLO   ")).alias("trim"),
    lpad(lit("HELLO"),3, " ").alias("lpad"),
    rpad(lit("HELLO"),10, " ").alias("rpad")).show(5)

+--------+---------+-----+----+----------+
|   ltrim|    rtrim| trim|lpad|      rpad|
+--------+---------+-----+----+----------+
|HELLO   |    HELLO|HELLO| HEL|HELLO     |
|HELLO   |    HELLO|HELLO| HEL|HELLO     |
|HELLO   |    HELLO|HELLO| HEL|HELLO     |
|HELLO   |    HELLO|HELLO| HEL|HELLO     |
|HELLO   |    HELLO|HELLO| HEL|HELLO     |
+--------+---------+-----+----+----------+
only showing top 5 rows



### 정규 표현식

문자열의 존재 여부를 확인하거나 일피하는 모든 문자열을 치환할 때는 보통 정규 표현식을 사용한다.

In [48]:
from pyspark.sql.functions import regexp_replace

regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
    regexp_replace(col("Description"),regex_string,"COLOR").alias("color_clean"),
    col("Description")).show(5)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
|CREAM CUPID HEART...|CREAM CUPID HEART...|
|KNITTED UNION FLA...|KNITTED UNION FLA...|
|COLOR WOOLLY HOTT...|RED WOOLLY HOTTIE...|
+--------------------+--------------------+
only showing top 5 rows



* translate함수를 사용해서 치환

In [51]:
from pyspark.sql.functions import translate, regexp_extract

In [50]:
df.select(translate(col("Description"),"LEEF","1337"),col("Description")).show(4)

+----------------------------------+--------------------+
|translate(Description, LEEF, 1337)|         Description|
+----------------------------------+--------------------+
|              WHIT3 HANGING H3A...|WHITE HANGING HEA...|
|               WHIT3 M3TA1 1ANT3RN| WHITE METAL LANTERN|
|              CR3AM CUPID H3ART...|CREAM CUPID HEART...|
|              KNITT3D UNION 71A...|KNITTED UNION FLA...|
+----------------------------------+--------------------+
only showing top 4 rows



L=1,E=3,T=7로 치환되었다.

* 처음 나타난 문자열 추출

In [54]:
regex_string = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(regexp_extract(col("Description"),regex_string,1).alias("color_clean"),
         col("Description")).show(5)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
|           |CREAM CUPID HEART...|
|           |KNITTED UNION FLA...|
|        RED|RED WOOLLY HOTTIE...|
+-----------+--------------------+
only showing top 5 rows



* contains메서드 : 값의 존재 여부를 확인

In [55]:
from pyspark.sql.functions import instr

In [57]:
containBlack = instr(col("Description"),"BLACK") >=1
containWhite = instr(col("Description"),"WHITE") >=1

df.withColumn("hasSimpleColor",containBlack | containWhite).where("hasSimpleColor").select("Description").show(3,False) 

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



파이썬은 인수의 개수가 동적으로 변하는 상황을 아주 쉽게 해결할 수 있다. 
* locate 함수 이용

In [62]:
from pyspark.sql.functions import expr, locate

In [58]:
simpleColors =["black",'white','red','green','blue']

In [69]:
def color_locator(column,color_string):
    return locate(color_string.upper(),column).cast("boolean").alias("is_"+color_string)

In [70]:
selectdColumns = [color_locator(df.Description,c) for c in simpleColors]
selectdColumns.append(expr("*"))

In [72]:
df.select(*selectdColumns).where(expr("is_white OR is_red")).select("Description").show(3,False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



### 날짜와 타임스탬프 데이터 타입 다루기

두 가지 종류의 사긴 관련 정보만 집중적으로 관리한다. 하나는 달력 형태의 날짜(data)이고, 다른 하나는 날짜와 시간 정보를 모두 가지는 타임스탬프이다. inferSchema옵션을 활성화된 경우 날짜와 타임스탬프를 포함해 컬럼의 데이터 타입을 최대한 정확하게 식별하려 시도한다.

In [73]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



TimestampType 클래스는 초 단위 정밀도까지만 지원한다. 그러므로 밀리세컨드나 마이크로세컨드 단뒤를 다룬다면 Long 데이터 타입으로 데이터를 변환해 처리하는 우회 정책을 사용해야 한다. 그 이상의 정밀도는 TimestampType으로 변환될 때 제거된다.

In [74]:
from pyspark.sql.functions import current_date, current_timestamp

In [75]:
dateDF = spark.range(10)\
        .withColumn("today",current_date())\
        .withColumn("now",current_timestamp())

In [76]:
dateDF.createOrReplaceTempView('dateTable')

In [77]:
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



 * date_sum(), date_add() : 컬럼과 더하거나 뺄 날짜 수를 인수로 전달

In [79]:
from pyspark.sql.functions import date_add, date_sub

In [80]:
dateDF.select(date_sub(col("today"),5),date_add(col("today"),5)).show()

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
|        2021-05-05|        2021-05-15|
+------------------+------------------+



* datediff : 두 날짜 사이의 일 수를 반환하는 함수
* months_between : 두 날짜 사이의 개월 수 반환

In [83]:
from pyspark.sql.functions import datediff, months_between, to_date

In [84]:
dateDF.withColumn("week_ago",date_sub(col("today"),7))\
    .select(datediff(col("week_ago"),col("today"))).show()

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
|                       -7|
+-------------------------+



* to_date : 문자열을 날짜로 변환

In [86]:
spark.range(5).withColumn("date",lit("2017-01-01"))\
    .select(to_date(col('date'))).show()

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
|     2017-01-01|
|     2017-01-01|
|     2017-01-01|
|     2017-01-01|
+---------------+



스파크는 날짜를 파싱할 수 없다면 에러 대신 null 값을 반환한다. 그러므로 다단계 처리 파이프라인에서는 조금 까다로울 수 있다. 

In [88]:
dateDF.select(to_date(lit("2016-20-12")),to_date(lit("2017-12-11"))).show(1)

+---------------------+---------------------+
|to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
|                 null|           2017-12-11|
+---------------------+---------------------+
only showing top 1 row



문제를 해결하기 위해 to_date 함수와 to_timestamp함수를 사용 

In [89]:
dateFormat = "yyyy-dd-MM"

In [91]:
cleanDataDF = spark.range(1).select(
        to_date(lit("2017-12-11"), dateFormat).alias("date"),
        to_date(lit("2017-20-12"),dateFormat).alias("date2")
)
cleanDataDF.createOrReplaceTempView("dateTable2")


In [93]:
cleanDataDF.show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



### null 값 다루기

DataFrane에서 빠져 있거나 비어있는 데이터를 표현할 때는 항상 null 값을 사용하는 것이 좋다.<br>
DataFrame의 하위 패키지인 .na를 사용하는 것이 DataFrame에서 null값을 다루는 기본 방식이다.

null 값을 다루는 두 가지 방범이 있다. 명식적으로 null값을 제거하거나, 전역 또는 컬럼 단위로 null 값을 특정 값으로 채워 넣는 것이다.

* coalesce : 여러 컬럼 중 null이 아닌 첫번 째 값을 반환. 모든 컬럼이 null이 아닌 값을 가지는 경우 첫번 째 컬럼의 값을 반환

In [94]:
from pyspark.sql.functions import coalesce

In [96]:
df.select(coalesce(col("Description"),col("CustomerID"))).show()

+---------------------------------+
|coalesce(Description, CustomerID)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



SQL함수
* ifnull : 첫번째 값이 null이면 두 번째 값을 반환. 첫 번째 값이 null이 아니면 첫 번째 값을 반환
* nulllf : 두 값이 같으면 null을 반환. 두 값이 다르면 첫 번째 값을 반환 
* nvl : 첫번째 값이 null이면 두 번째 값을 반환. 첫 번째 값이 null이 아니면 첫 번째 값을 반환
* nvl2 : 첫 번째 값이 null이 아니면 두 번째 값을 반환. 그리고 첫 번째 값이 nulld이면 세 번째 인수로 지정된 값을 반환

* drop : null값을 가진 로우를 제거하는 가장 간단한 함수

drop메서드의 인수로 any를 지정한 경우 로우의 컬럼값 중 하나라도 null값을 가지면 해당 로우를 제거. all을 지정한 경우 모든 컬럼값이 null이거나 Nan인 경우에만 해당 로우를 제거

In [98]:
df.na.drop()
df.na.drop("any")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

배열 형태의 컬럼을 인수로 전달해 적용할 수도 있다.

In [99]:
df.na.drop("all",subset = ['StockCode','InvoiceNo'])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

* fill : 하나 이상의 컬럼을 특정 값으로 채울 수 있다. 채워 넣을 값과 컬럼 집합으로 구성된 맵을 인수로 사용한다.

In [100]:
df.na.fill("All Null values become this string")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [102]:
df.na.fill("all",subset=["StockCode","InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [103]:
fill_cols_vals = {"StockCode":5,"Description" : "No value"}

In [104]:
df.na.fill(fill_cols_vals)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

* replace : drop과 fill 외에도 null값을 유연하게 대처할 방법이 있다. 조건에 따라 다른 값으로 replace하는 것이다. **변경하고자 하는 값과 원래 값의 데이터 타입이 같아야 한다**

In [105]:
df.na.replace([""],["UNKNOWN"],"Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

### 복합 데이터 타입 다루기

* 구조체 : DataFrame 내부의 DataFrame으로 생각할 수 있다. 퀴리문에서 다수의 컬럼을 괄호로 묶어 구조체를 만들 수 있다.

In [110]:
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description","InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

In [114]:
complexDF.printSchema()

root
 |-- complex: struct (nullable = false)
 |    |-- Description: string (nullable = true)
 |    |-- InvoiceNo: string (nullable = true)



In [115]:
complexDF.select("complex.*")

DataFrame[Description: string, InvoiceNo: string]

* 배열 
    * split : 구분자를 인수로 전달해 배열로 변환한다.

In [116]:
from pyspark.sql.functions import split

In [117]:
df.select(split(col("Description")," ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



split 함수는 스파크에서 복합 데이터 타입을 마치 또 다른 컬럼처럼 다룰 수 있는 매우 강력한 기능이다.

In [118]:
df.select(split(col("Description"), " ").alias("array_col")).selectExpr("array_col[0]").show()

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
|       CREAM|
|     KNITTED|
|         RED|
|         SET|
|       GLASS|
|        HAND|
|        HAND|
|    ASSORTED|
|     POPPY'S|
|     POPPY'S|
|   FELTCRAFT|
|       IVORY|
|         BOX|
|         BOX|
|         BOX|
|        HOME|
|        LOVE|
|      RECIPE|
+------------+
only showing top 20 rows



* 배열의 크기를 조회해 배열의 길이를 알 수 있다.

In [120]:
from pyspark.sql.functions import size

In [121]:
df.select(size(split(col("Description"), " "))).show(2)

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows



* array_contains : 배열에 특정 값이 존재하는지 확인

In [122]:
from pyspark.sql.functions import array_contains

In [123]:
df.select(array_contains(split(col("Description"), " "),"WHITE")).show(2)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows



* explode : 배열 타입의 컬럼을 입력 받는다. 그리고 컬럼의 배열값에 포홤된 모든 값을 로우로 변환한다. 나머지 컬럼값은 중복되어 표시

In [125]:
from pyspark.sql.functions import explode

In [131]:
df.withColumn("splitted",split(col("Description"), " "))\
    .withColumn("exploded", explode(col("splitted")))\
    .select("Description","InvoiceNo","exploded").show(5)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
|WHITE HANGING HEA...|   536365| T-LIGHT|
|WHITE HANGING HEA...|   536365|  HOLDER|
+--------------------+---------+--------+
only showing top 5 rows



* map : 컬럼의 키-값 쌍을 이용해 생성한다. 

In [139]:
from pyspark.sql.functions import create_map

df.select(create_map(lit(1),col("Description")).alias("complex_map")).show()

+--------------------+
|         complex_map|
+--------------------+
|[1 -> WHITE HANGI...|
|[1 -> WHITE METAL...|
|[1 -> CREAM CUPID...|
|[1 -> KNITTED UNI...|
|[1 -> RED WOOLLY ...|
|[1 -> SET 7 BABUS...|
|[1 -> GLASS STAR ...|
|[1 -> HAND WARMER...|
|[1 -> HAND WARMER...|
|[1 -> ASSORTED CO...|
|[1 -> POPPY'S PLA...|
|[1 -> POPPY'S PLA...|
|[1 -> FELTCRAFT P...|
|[1 -> IVORY KNITT...|
|[1 -> BOX OF 6 AS...|
|[1 -> BOX OF VINT...|
|[1 -> BOX OF VINT...|
|[1 -> HOME BUILDI...|
|[1 -> LOVE BUILDI...|
|[1 -> RECIPE BOX ...|
+--------------------+
only showing top 20 rows



In [144]:
df.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("complex_map['WHITE METAL LANTERN']").show(2)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
+--------------------------------+
only showing top 2 rows



In [145]:
df.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("explode(complex_map)").show(2)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows



### JSON 다루기

스파크에서는 문자열 형태의 JSON을 직접 조작할 수 있으며, JSON을 파싱하거나 JSON객체로 만들 수 있다.  

In [153]:
jsonDF = spark.range(1).selectExpr(""" '{"myJsonKey": {"myJsonValue" : [1,2,3]}}' as jsonString""")

* get_json_object : JSON 객체를 인라인 쿼리로 조뢰할 수 있다. 중

In [147]:
from pyspark.sql.functions import get_json_object, json_tuple

In [154]:
jsonDF.select(get_json_object(col("jsonString"),"$.myJsonKey.myJsonValue[1]").alias("column"),
             json_tuple(col("jsonString"),"myJsonKey")).show(2)

+------+--------------------+
|column|                  c0|
+------+--------------------+
|     2|{"myJsonValue":[1...|
+------+--------------------+



* to_json : StructType을 JSON 문자열로 변경

In [155]:
from pyspark.sql.functions import to_json

In [156]:
df.selectExpr("(InvoiceNo, Description) as myStruct").select(to_json(col("myStruct")))

DataFrame[to_json(myStruct): string]

to_json함수에 JSON 데이터소스와 동일한 형태의 딕셔너리를 파라미터로 사용할 수 있다. 그리고 from_json함수를 사용해 JSON 문자열을 다시 객체로 변환할 수 있다. from_json 함수는 파라미터로 스키마를 지정해야한다.

In [160]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *


In [162]:
parseSchema = StructType((
    StructField("InvoiceNo",StringType(),True),
    StructField("Description",StringType(),True)))

In [163]:
df.selectExpr("(InvoiceNo,Description) as myStruct")\
    .select(to_json(col('myStruct')).alias("newJSON"))\
    .select(from_json(col("newJSON"),parseSchema),col("newJSON")).show()

+--------------------+--------------------+
|  from_json(newJSON)|             newJSON|
+--------------------+--------------------+
|[536365, WHITE HA...|{"InvoiceNo":"536...|
|[536365, WHITE ME...|{"InvoiceNo":"536...|
|[536365, CREAM CU...|{"InvoiceNo":"536...|
|[536365, KNITTED ...|{"InvoiceNo":"536...|
|[536365, RED WOOL...|{"InvoiceNo":"536...|
|[536365, SET 7 BA...|{"InvoiceNo":"536...|
|[536365, GLASS ST...|{"InvoiceNo":"536...|
|[536366, HAND WAR...|{"InvoiceNo":"536...|
|[536366, HAND WAR...|{"InvoiceNo":"536...|
|[536367, ASSORTED...|{"InvoiceNo":"536...|
|[536367, POPPY'S ...|{"InvoiceNo":"536...|
|[536367, POPPY'S ...|{"InvoiceNo":"536...|
|[536367, FELTCRAF...|{"InvoiceNo":"536...|
|[536367, IVORY KN...|{"InvoiceNo":"536...|
|[536367, BOX OF 6...|{"InvoiceNo":"536...|
|[536367, BOX OF V...|{"InvoiceNo":"536...|
|[536367, BOX OF V...|{"InvoiceNo":"536...|
|[536367, HOME BUI...|{"InvoiceNo":"536...|
|[536367, LOVE BUI...|{"InvoiceNo":"536...|
|[536367, RECIPE B...|{"InvoiceN

### 사용자 정의 함수

스파크의 가장 강력한 기능 중 하나는 사용자 정의 함수를 사용할 수 있다. UDF는 파이썬이나 스칼라 그리고 외부 라이브러리를 사용해 사용자가 원하는 형태로 트랜스포메이션을 만들 수 있게 한다. UDF는 하나 이상의 컬ㄹ럼을 입력으로 받고, 반환할 수 있다. 스파크 UDF는 여러 가지 프로그래밍 언어로 개발할 수 있다. UDF는 레코드별로 데이터를 처리하는 함수이기 때문에 독특한 포맷이나 도메인에 특화된 언어를 사용하지 않는다. 이러한 UDF는 기본적으로 특정 SparkSession이나 Context에서 사용할 수 있도록 임시 함수 형태로 등록된다. 

In [164]:
udfExampleDF = spark.range(5).toDF("num")

In [167]:
def power3(double_value):
    return double_value ** 3
power3(2.0)

8.0

1. 생성된 함수를 사용할 수 있도록 스파크에 등록
2. 스파크는 드라이버에서 함수를 직렬화하고 네트워크를 통해 모든 익스큐터 프로세스로 전달
        스파크와 자바로 함수를 작성했다면 JVM환경에서만 사용할 수 있다. 따라서 스파크 내장 함수가 제공하는 코드생성 기능의 장점을 활용할 수 없어 약간의 성능 저하가 발생한다. 그리고 많은 객체를 생성하거나 사용해도 성능 문제가 발생한다.
3. 스파크는 워커 노드에 파이썬 프로세스를 실행하고 파이썬이 이해할 수 있는 포맷으로 모든 데이터를 직렬화한다. 
4. 파이썬 프로세스에 있는 데이터의 로우마다 함수를 샐행하고 마지막으로 JVM과 스파크에 처리 결과를 반환한다.


![](./IMG_7DE6D1CE44D2-1.jpeg)

        파이썬 프로세스를 시작하는 부하도 크지만, 진짜 부하는 파이썬으로 데이터를 전달하기 위해 직렬화하는 과정에서 발생한다. 이특성은 두 가지 문제점을 만들어낸다. 첫째, 직렬화에 큰 부하가 발생한다. 둘째, 데이터가 파이썬으로전달되면 스파크에서 워커메모리를 관리할 수 없다. 그러므로 JVM과 파이썬이 동일한 머신에서 메모리 경합을 하면 자원에 제약이 생겨 워커가 비정상적으로 종료될 가능성이 있다. 스칼라로 함수를 개발하는 것은 시간이 오래 걸리지 않으며 무엇보다 파이썬에서도 사용할 수 있으므로 자바나 스칼라로 사용자 정의 함수를 작성하는 것이 좋다

In [168]:
from pyspark.sql.functions import udf

In [169]:
power3udf = udf(power3)

In [170]:
udfExampleDF.select(power3udf(col("num"))).show()

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
|          8|
|         27|
|         64|
+-----------+



* 사용자 정의함수 SQL함수로 등록 : 모든 프로그래밍 언어와 SQL에서 사용자 정의 함수를 사용할 수 있다.

In [172]:
from pyspark.sql.types import IntegerType, DoubleType

In [173]:
spark.udf.register("power3py",power3,DoubleType())

<function __main__.power3(double_value)>

In [175]:
udfExampleDF.selectExpr("power3py(num)").show(2)

+-------------+
|power3py(num)|
+-------------+
|         null|
|         null|
+-------------+
only showing top 2 rows

